In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
class fsaClassifier:
    def __init__(self, xTrain, yTrain, xValid, yValid, nIter=500, eta=0.0001, k=10, s=0.001, miu=100.):
        self.xTrain=xTrain
        self.yTrain=yTrain
        self.xValid=xValid
        self.yValid=yValid
        self.s=s
        self.miu=miu
        self.nIter=nIter
        self.k=k
        self.eta=eta
        self.preProcess()
        self.M=self.xTrain.shape[1]
        self.w=np.zeros(self.xTrain.shape[1])

    def preProcess(self):
        xStd=np.std(self.xTrain, axis=0)
        mask=(xStd!=0.)
        self.xTrain=self.xTrain[:, mask]
        meanX=np.mean(self.xTrain, axis=0)
        stdX=np.std(self.xTrain, axis=0)
        self.xTrain=(self.xTrain-meanX)/stdX
        self.xValid=self.xValid[:, mask]
        self.xValid=(self.xValid-meanX)/stdX

        self.xTrain=np.insert(self.xTrain, 0, 1., axis=1)
        self.xValid=np.insert(self.xValid, 0, 1., axis=1)

        self.yTrain[self.yTrain==0.]=-1.
        self.yValid[self.yValid==0.]=-1.
        
    def gradient(self):
        wx=np.sum(self.xTrain*self.w, axis=1)
        ywx=wx*self.yTrain
        temp=2.*(ywx-1.)/(1.+(ywx-1.)*(ywx-1.))
        temp=temp*self.yTrain
        temp[ywx>1.]=0.
        grad=np.sum((self.xTrain).T*temp, axis=1)+2.*self.s*self.w
        return grad

    def update(self, i):
        grad=self.gradient()
        self.w-=self.eta*grad
        self.fsa(i)

    def fsa(self, i):
        mi=self.k+(self.M-self.k)*max(0.,
                (self.nIter-2*i)/(2*i*self.miu+self.nIter))
        mi=int(mi)
        #print(i, 'Num Features:', mi)
        wAbs=np.absolute(self.w)
        wAbsSort=np.argsort(wAbs)
        wAbsSort=wAbsSort[-mi:]
        self.w=self.w[wAbsSort]
        self.xTrain=(self.xTrain.T[wAbsSort]).T
        self.xValid=(self.xValid.T[wAbsSort]).T

    def train(self):
        losses=np.zeros(self.nIter)
        
        for i in range(self.nIter):
            self.update(i)
            wx = np.sum(self.xTrain*self.w, axis=1)
            ywx = wx*self.yTrain
            temp = np.log(1.+(ywx-1.)*(ywx-1.))
            temp[ywx>1.]=0.
            losses[i]=np.sum(temp)+self.s*np.sum(self.w*self.w)
        
        wx = np.sum(self.xTrain*self.w, axis=1)
        pred=np.ones(self.yTrain.shape[0])
        pred[wx<0.]=-1
        trainError = 1. - np.mean(pred==self.yTrain)

        wx=np.sum(self.xValid*self.w, axis=1)
        pred=np.ones(self.yValid.shape[0])
        pred[wx<0.]=-1
        testError = 1.-np.mean(pred==self.yValid)  
        
        return losses, trainError, testError

In [ ]:
def graphLosses(losses):
    itrs = [x for x in range(len(losses))]
    plt.plot(itrs, losses, linestyle='-', marker='o', color='r', label='loss')
    plt.title('Training Loss vs Iteration Number, k = 30')
    plt.xlabel('Iteration')
    plt.ylabel('Training Loss')
    plt.legend()
    plt.show()

In [ ]:
def graphErrors(trainErrors, testErrors, K):
    plt.plot(K, trainErrors, linestyle='-', marker='o', color='r', label='Train')
    plt.plot(K, testErrors, linestyle='-', marker='o', color='b', label='Test')
    plt.grid(True)
    plt.xticks(K)
    plt.title('Misclassification Error vs Number of Features')
    plt.xlabel('Number of Features (k)')
    plt.ylabel('Misclassification Error (%)')
    plt.legend()
    plt.show()

In [ ]:
def genTable(trainErrors, testErrors, K):
    trainErrors = trainErrors * 100
    testErrors = testErrors * 100
    
    errorTable = pd.DataFrame({"Training Error (%)":[0, 0, 0, 0, 0], "Test Error (%)":[0, 0, 0, 0, 0]}, index=K)
    errorTable.index.name = "k"

    for (k, train, test) in zip(K, trainErrors, testErrors):
        errorTable.loc[k, 'Training Error (%)'] = round(train, 2)
        errorTable.loc[k, 'Test Error (%)'] = round(test, 2)
    
    return errorTable

In [ ]:
K = [10, 30, 100, 300, 500]
etas=[0.0001, 0.0001, 0.0001, 0.0001, 0.0001]

# Part A

In [ ]:
xTrain = np.loadtxt('../data/Gisette/gisette_train.data')
yTrain = np.loadtxt('../data/Gisette/gisette_train.labels')
xValid = np.loadtxt('../data/Gisette/gisette_valid.data')
yValid = np.loadtxt('../data/Gisette/gisette_valid.labels')

In [ ]:
#etas=[0.005, 0.0001, 0.0001, 0.0001, 0.0001]

trainErrors=np.zeros(len(K))
testErrors=np.zeros(len(K))
figureIndex=0

for i in range(len(K)):
    model = fsaClassifier(xTrain, yTrain, xValid, yValid, eta=etas[i],
            k=K[i])
    losses, misclassTr, misclassVal=model.train()
    
    if K[i] == 30:
        graphLosses(losses)
        
    trainErrors[i] = misclassTr
    testErrors[i] = misclassVal

In [ ]:
graphErrors(trainErrors, testErrors, K)
errorGis = genTable(trainErrors, testErrors, K)

# Part B

In [ ]:
xTrain=np.genfromtxt('../data/dexter/dexter_train.csv', delimiter=',')
yTrain=np.loadtxt('../data/dexter/dexter_train.labels')
xValid=np.genfromtxt('../data/dexter/dexter_valid.csv', delimiter=',')
yValid=np.loadtxt('../data/dexter/dexter_valid.labels')

In [ ]:
#etas=[0.0005, 0.001, 0.001, 0.001, 0.0001]

trainErrors=np.zeros(len(K))
testErrors=np.zeros(len(K))

for i in range(len(K)):
    model = fsaClassifier(xTrain, yTrain, xValid, yValid, eta=etas[i],
            k=K[i])
    losses, misclassTr, misclassVal=model.train()
    
    if K[i] == 30:
        graphLosses(losses)
        
    trainErrors[i] = misclassTr
    testErrors[i] = misclassVal

In [ ]:
graphErrors(trainErrors, testErrors, K)
errorDex = genTable(trainErrors, testErrors, K)

# Part C

In [ ]:
xTrain=np.loadtxt('../data/MADELON/madelon_train.data')
yTrain=np.loadtxt('../data/MADELON/madelon_train.labels')
xValid=np.loadtxt('../data/MADELON/madelon_valid.data')
yValid=np.loadtxt('../data/MADELON/madelon_valid.labels')

In [ ]:
#etas=[0.001, 0.0001, 0.0005, 0.0001, 0.0001]

trainErrors=np.zeros(len(K))
testErrors=np.zeros(len(K))

for i in range(len(K)):
    model = fsaClassifier(xTrain, yTrain, xValid, yValid, eta=etas[i],
            k=K[i])
    losses, misclassTr, misclassVal=model.train()
    
    if K[i] == 30:
        graphLosses(losses)
        
    trainErrors[i] = misclassTr
    testErrors[i] = misclassVal

In [ ]:
graphErrors(trainErrors, testErrors, K)
errorMad = genTable(trainErrors, testErrors, K)

In [ ]:
errorTableFull = pd.concat([errorGis, errorDex, errorMad])
errorTableFull